# Imports

In [1]:
import os
Smart_charging_path = os.getcwd()
from os.path import dirname, join, exists
import win32com.client
dssObj = win32com.client.Dispatch("OpenDSSEngine.DSS")
dssObj = win32com.client.Dispatch("OpenDSSEngine.DSS")
dssObj.AllowForms = False # Disable progress bar
dssText = dssObj.Text
dssCircuit = dssObj.ActiveCircuit
dssSolution = dssCircuit.Solution
dssElem = dssCircuit.ActiveCktElement
dssBus = dssCircuit.ActiveBus
DssMonitors = dssCircuit.Monitors
import pandas as pd # type: ignore
import datetime as dt
import numpy as np # type: ignore
from pymoo.problems.functional import FunctionalProblem # type: ignore
from pymoo.optimize import minimize # type: ignore
import plotly.express as px # type: ignore
import warnings
import random
random.seed(10)
warnings.filterwarnings("ignore")
import plotly.graph_objects as go # type: ignore
from pymoo.visualization.scatter import Scatter # type: ignore
from pymoo.decomposition.asf import ASF # type: ignore
import matplotlib.pyplot as plt

# Paths

In [2]:
Grid_model_path = join(dirname(Smart_charging_path), "1_Grid_model")
Results_path = join(Smart_charging_path, 'Results')

# General params

In [3]:
run_datetime = dt.datetime(dt.date.today().year, dt.date.today().month, dt.date.today().day)
sim_period = pd.timedelta_range(0, dt.timedelta(days=1), freq='5T', closed='left')
sim_datetimes = pd.date_range(run_datetime, run_datetime + dt.timedelta(days=1),freq='5T', inclusive='left')
load_names = ['load_' + str(i) for i in range(1, 118)]
main_buses_names = [str(i) for i in range(1, 47)] + [str(i) for i in range(49, 52)] + \
    ['56'] + ['61'] + ['66'] + ['73'] + ['80'] + ['87'] + ['96'] + ['106'] + ['117'] + \
    ['127'] + ['137'] + ['151'] + ['169'] + ['188'] + ['206'] + \
    ['228'] + ['253'] + ['278'] + ['304']
CPs_names = ['CP_' + str(i) for i in range(1, 4)]
PVs_names = ['PV_' + str(i) for i in range(1, 13)]

# Loads data

In [4]:
P_loads = pd.DataFrame(index=sim_datetimes)
for load in load_names:
    P_loads[load] = pd.read_csv(join(Grid_model_path, "Load_profiles", load.split('_')[0] + load.split('_')[1] + '_5min.csv'), header=None).values

In [5]:
# px.line(P_loads)

# CPs data

In [6]:
max_power = 11

In [7]:
P_CPs = pd.DataFrame(
    index=sim_datetimes,
    columns=CPs_names,
    data=np.stack([np.concatenate([np.repeat(0, 12*12), np.repeat(max_power, 12), np.repeat(0, 11*12)]),
                   np.concatenate([np.repeat(0, 9*12), np.repeat(max_power, 12), np.repeat(0, 14*12)]),
                   np.concatenate([np.repeat(0, 14*12), np.repeat(max_power, 2*12), np.repeat(0, 8*12)])],
                   axis=1)
)

In [ ]:
px.line(P_CPs)

# PV data

In [9]:
PV_date = '2024-05-28'

In [10]:
P_PVs = pd.read_csv(join(Grid_model_path, 'PV_data', 'PV.csv'), index_col=0, parse_dates=True)[PV_date]

In [11]:
P_PVs = pd.DataFrame(
    index=sim_datetimes,
    columns=PVs_names,
    data=np.tile(P_PVs, len(PVs_names)) * 4
)

In [ ]:
px.line(P_PVs)

# Compilation of the main .dss file of the grid model

In [13]:
dssText.Command = "compile " + join(Grid_model_path, "Master.dss")

# Definition of the function that performs the grid simulation for a given timestamp 

In [14]:
def Grid_sim(P_loads_t):
    def Grid_sim_fixed_loads(P_CPs_t, P_PVs_t):
        for load in load_names:                 # Update the load power consumption
            dssText.Command = "Edit Load." + load + " kW=" + str(max(P_loads_t[load], 1e-10))
        for CP in CPs_names:
            dssText.Command = "Edit Load." + CP + " kW=" + str(max(P_CPs_t[CP], 1e-10))
        for PV in PVs_names:
            dssText.Command = "Edit Load." + PV + " kW=" + str(min(-P_PVs_t[PV], -1e-10))
        dssSolution.Solve()                     # Solve a power flow simulation
                       # Select the voltage source to get some results at it
        # P_source = np.array([- dssElem.Powers[0], - dssElem.Powers[2], - dssElem.Powers[4]])
        total_buses = len(dssCircuit.AllBusNames)
        all_V = pd.Series(
                index=dssCircuit.AllBusNames,
                data = np.array(dssCircuit.AllBusVmagPu).reshape(total_buses, 3).min(axis=1)
                )        # Get all voltages in the main buses 
        PUI = np.array([])
        dssCircuit.Lines.Name = 'LINE161'
        current = dssElem.CurrentsMagAng
        I = np.array([current[0], current[2], current[4]])
        PUI = np.append(PUI, np.max(np.abs(I - np.repeat(np.mean(I), 3))) / np.mean(I) * 100)
        dssCircuit.Lines.Name = 'LINE3163'
        current = dssElem.CurrentsMagAng
        I = np.array([current[0], current[2], current[4]])
        PUI = np.append(PUI, np.max(np.abs(I - np.repeat(np.mean(I), 3))) / np.mean(I) * 100)
        dssCircuit.Lines.Name = 'LINE2690'
        current = dssElem.CurrentsMagAng
        I = np.array([current[0], current[2], current[4]])
        PUI = np.append(PUI, np.max(np.abs(I - np.repeat(np.mean(I), 3))) / np.mean(I) * 100)
        return all_V[main_buses_names], PUI
    return Grid_sim_fixed_loads

# Parameters

In [15]:
N1 = 0
N2 = 0 
alpha = 1.1
U_size = (3 + 12) * (N2 - N1 + 1) # Minpulated variables size
Alpha = np.array([alpha**(N2 - j) for j in range(N1, N2+1)])
X_opt_cols = [e + '_' + str(i) for i in range(N1+1, N2+2) for e in CPs_names+PVs_names]

# Objective functions

## Obj 1

$$ \underset{\underset{1 \leq i \leq 3,\ 1 \leq p \leq 12}{P_{\mathrm{CP}_i}(t),\ P_{\mathrm{PV}_{p}}(t)}}{min} f_1(t) = \sum_{i = 1}^{3} \left[\mathrm{PUI}_{\mathrm{bus\_CP}_i}(t)\right]^2 $$

In [16]:
def objective_1(P_loads_t):
    def objective_1_fixed_loads(U):
        U_df = pd.Series(index=CPs_names+PVs_names, data=U)
        P_CPs_opt_t = U_df[CPs_names]
        P_PVs_opt_t = U_df[PVs_names]
        for load in load_names:                 # Update the load power consumption
            dssText.Command = "Edit Load." + load + " kW=" + str(max(P_loads_t[load], 1e-10))
        for CP in CPs_names:
            dssText.Command = "Edit Load." + CP + " kW=" + str(max(P_CPs_opt_t[CP], 1e-10))
        for PV in PVs_names:
            dssText.Command = "Edit Load." + PV + " kW=" + str(min(-4 * P_PVs_opt_t[PV], -1e-10))
        dssSolution.Solve()                     # Solve a power flow simulation
        PUI = np.array([])
        dssCircuit.Lines.Name = 'LINE161'
        current = dssElem.CurrentsMagAng
        I = np.array([current[0], current[2], current[4]])
        PUI = np.append(PUI, np.max(np.abs(I - np.repeat(np.mean(I), 3))) / np.mean(I) * 100)
        dssCircuit.Lines.Name = 'LINE3163'
        current = dssElem.CurrentsMagAng
        I = np.array([current[0], current[2], current[4]])
        PUI = np.append(PUI, np.max(np.abs(I - np.repeat(np.mean(I), 3))) / np.mean(I) * 100)
        dssCircuit.Lines.Name = 'LINE2690'
        current = dssElem.CurrentsMagAng
        I = np.array([current[0], current[2], current[4]])
        PUI = np.append(PUI, np.max(np.abs(I - np.repeat(np.mean(I), 3))) / np.mean(I) * 100)
        return np.square(PUI).sum()
    return objective_1_fixed_loads

## Obj 2

$$ \underset{\underset{1 \leq p \leq 12}{\ P_{\mathrm{PV}_{p}}(t)}}{min} f_2(t) =  \sum_{p = 1}^{12} \left[P_{PV_{\mathrm{actual}}}(t) - P_{PV_{p}}(t)\right]^2$$

In [17]:
def objective_2(P_PVs_t):
    def objective_2_compared_to_actual_PV(U):
        U_df = pd.DataFrame(columns=CPs_names+PVs_names, data=U.reshape(N2 - N1 + 1, int(U_size / (N2 - N1 + 1))))
        P_PVs_opt_t = U_df[PVs_names]
        return np.sum((P_PVs_t.values - P_PVs_opt_t.values)** 2, axis=1)
    return objective_2_compared_to_actual_PV

# Termination

In [18]:
from pymoo.termination import get_termination # type: ignore
from pymoo.core.termination import TerminateIfAny # type: ignore

myTermination = TerminateIfAny(get_termination("n_gen", 100), get_termination("moo"))

# Algorithm choice: Non-dominated Sorting Genetic Algorithm (NSGA2)

In [19]:
from pymoo.algorithms.moo.nsga2 import NSGA2 # type: ignore

algorithm = NSGA2(pop_size=100)

# Test cell

In [20]:
# ts = sim_datetimes[144]
# P_loads_t = P_loads.loc[ts]
# P_CPs_t = P_CPs.loc[ts]
# P_PVs_t = P_PVs.loc[ts]
# U = np.concatenate([P_CPs_t, P_PVs_t])

# Initialize

In [21]:
P_CPs_opt = pd.DataFrame(columns=CPs_names)
P_PVs_opt = pd.DataFrame(columns=PVs_names)
V = pd.DataFrame(columns=main_buses_names)
PUI = pd.DataFrame(columns=['PUI_bus_CP1', 'PUI_bus_CP2', 'PUI_bus_CP3'])

# Run Smart Control loop

In [ ]:
# %%time
for ts in sim_datetimes:
    print("| timestamp: " + str(ts) + (56 - len(" timestamp: " + str(ts))) * " " + "|") 
    # Input actual values
    P_loads_t = P_loads.loc[ts]
    P_CPs_t = P_CPs.loc[ts]
    P_PVs_t = P_PVs.loc[ts]
    # If there is no EV charging demand no need for optimization, the total PV power will be injected without control
    if np.all(P_CPs_t == 0): 
        print("| No optimization is needed" + (56 - len(" No optimization is needed")) * " " + "|") 
        print('+' + '-' * 56 + '+')
        P_CPs_opt_t, P_PVs_opt_t = P_CPs_t, P_PVs_t
    # Else run the optimization for controlling the CP and PV powers
    else:
        print("| Running the optimization" + (56 - len(" Running the optimization")) * " " + "|") 
        # Define lower and upper bounds of the control variables
        Lb_CP, Ub_CP = P_CPs_t.values * 0.9, P_CPs_t.values * 1.1
        Lb_PV, Ub_PV = np.repeat(0, 12), P_PVs_t.values
        Lb, Ub = np.concatenate([Lb_CP, Lb_PV]), np.concatenate([Ub_CP, Ub_PV])
        # Define the optimization problem
        problem = FunctionalProblem(n_var=U_size, 
                                    objs=[objective_1(P_loads_t), objective_2(P_PVs_t)],
                                    xl=Lb,
                                    xu=Ub,
                                    )
        # Run the optimization
        opt = minimize(problem, algorithm, seed=1, termination=myTermination, disp=True, verbose=True)
        X_star = pd.DataFrame(columns=X_opt_cols, data=opt.X)
        F_star = pd.DataFrame(columns=['f1', 'f2'], data=opt.F)
        # Save X and F
        X_F_res = pd.concat([X_star, F_star], axis=1)
        if not exists(join(Results_path, 'Control_results', "X_F_results")): os.makedirs(join(Results_path, 'Control_results', "X_F_results"))
        X_F_res.to_csv(join(Results_path, 'Control_results', "X_F_results", "X_F_res_" + ts.strftime("%H_%M") + ".csv"))
        # Plot and save f1_f2 fig
        plot = Scatter()
        plot.add(opt.F, facecolor="none", edgecolor="red")
        plot.figsize = (20,10)
        if not exists(join(Results_path, 'Control_results', "F1_F2_figures")): os.makedirs(join(Results_path, 'Control_results', "F1_F2_figures"))
        plot.save(join(Results_path, 'Control_results', "F1_F2_figures","f1_f2_" + ts.strftime("%H_%M") + ".jpg"))
        # MCDM of the choice of X_opt (Compromise solution [0.5,0.5])
        approx_ideal = opt.F.min(axis=0)
        approx_nadir = opt.F.max(axis=0)
        nF = (opt.F - approx_ideal) / (approx_nadir - approx_ideal)
        weights = np.array([0.5, 0.5])
        decomp = ASF()
        i_f = decomp.do(nF, 1/weights).argmin()
        comp_prog_res = X_F_res.iloc[i_f,:]
        # Plot and save f1_f2 fig
        plt.figure(figsize=(20,10), dpi=400)
        plt.scatter(opt.F[:, 0], opt.F[:, 1], s=30, facecolors='none', edgecolors='blue')
        plt.scatter(opt.F[i_f, 0], opt.F[i_f, 1], marker="x", color="red", s=200)
        plt.title("Objective Space")
        if not exists(join(Results_path, 'Control_results', "F1_F2_figures")): os.makedirs(join(Results_path, 'Control_results', "F1_F2_figures"))
        plt.savefig(join(Results_path, 'Control_results', "F1_F2_figures","f1_f2_" + ts.strftime("%H_%M") + ".jpg"))
        # Simulate with optimal values
        P_CPs_opt_t, P_PVs_opt_t = comp_prog_res[[CP + '_1' for CP in CPs_names]], comp_prog_res[[PV + '_1' for PV in PVs_names]]
        P_CPs_opt_t.index, P_PVs_opt_t.index = CPs_names, PVs_names
        print('+' + '-' * 56 + '+')
    P_CPs_opt.loc[ts], P_PVs_opt.loc[ts] = P_CPs_opt_t.values, P_PVs_opt_t.values
    V.loc[ts], PUI.loc[ts] = Grid_sim(P_loads.loc[ts])(P_CPs_opt_t, P_PVs_opt_t)
    # Saving results in the Hard disk
    # if not exists(join(Results_path, 'Control_results', "Sim_results")): os.makedirs(join(Results_path, 'Control_results', "Sim_results"))
    # P_CPs_opt.to_csv(join(Results_path, 'Control_results', "Sim_results", "P_CPs_opt.csv"))
    # P_PVs_opt.to_csv(join(Results_path, 'Control_results', "Sim_results", "P_PVs_opt.csv"))
    # V.to_csv(join(Results_path, 'Control_results', "Sim_results", "V.csv"))
    # PUI.to_csv(join(Results_path, 'Control_results', "Sim_results", "PUI.csv"))

# Results

In [24]:
P_CPs_opt = pd.read_csv(join(Results_path, 'Control_results', "Sim_results", "P_CPs_opt.csv"), index_col=0, parse_dates=True)
P_PVs_opt = pd.read_csv(join(Results_path, 'Control_results', "Sim_results", "P_PVs_opt.csv"), index_col=0, parse_dates=True)
V = pd.read_csv(join(Results_path, 'Control_results', "Sim_results", "V.csv"), index_col=0, parse_dates=True)
PUI = pd.read_csv(join(Results_path, 'Control_results', "Sim_results", "PUI.csv"), index_col=0, parse_dates=True)

## Phase unbalance

In [ ]:
px.line(PUI)

## PV power usage

In [ ]:
px.line(P_PVs_opt)

## Voltage drop

In [ ]:
px.line(V)